In [1]:
DEBUG = False

In [2]:
!pip install albumentations

In [3]:
# !pip install git+https://github.com/ildoonet/pytorch-gradual-warmup-lr.git

In [4]:
import os
import sys
sys.path = [
    '../input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master',
] + sys.path



In [5]:
import time
import skimage.io
import numpy as np
import pandas as pd
import cv2
import PIL.Image as Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SubsetRandomSampler, RandomSampler, SequentialSampler
# from warmup_scheduler import GradualWarmupScheduler
from efficientnet_pytorch import model as enet
import albumentations
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
from sklearn.metrics import cohen_kappa_score
from tqdm import tqdm_notebook as tqdm


# Configuration

In [6]:
DATASET_FOLDER_PATH: str = os.path.join(os.path.abspath('./Prostate_Cancer_Grade_Assessment'), 'dataset')
TRAINING_DATA_FOLDER: str = 'train-tiled-prostate-36x256x256'
TESTING_DATA_FOLDER: str = 'test-tiled-prostate-36x256x256'
TRAIN_DATA_CSV_PATH: str = os.path.join(DATASET_FOLDER_PATH, TRAINING_DATA_FOLDER, 'train.csv')
TEST_DATA_CSV_PATH: str = os.path.join(DATASET_FOLDER_PATH, TESTING_DATA_FOLDER, 'test.csv')

# data_dir = '../input/prostate-cancer-grade-assessment' # Prostate_Cancer_Grade_Assessment/dataset/train-tiled-prostate-36x256x256
df_train = pd.read_csv(TRAIN_DATA_CSV_PATH)
df_train = df_train[df_train.is_present == 1].reset_index(drop=True).copy()
print(len(df_train))

df_test = pd.read_csv(TEST_DATA_CSV_PATH)
df_test = df_test[df_test.is_present == 1].reset_index(drop=True).copy()
print(len(df_test))

TRAINING_DATA_FOLDER_PATH = os.path.join(DATASET_FOLDER_PATH, TRAINING_DATA_FOLDER)
TESTING_DATA_FOLDER_PATH = os.path.join(DATASET_FOLDER_PATH, TESTING_DATA_FOLDER)

kernel_type = 'how_to_train_effnet_b0_to_get_LB_0.86'

enet_type = 'efficientnet-b0'
fold = 0
tile_size = 256
image_size = 256
n_tiles = 36
batch_size = 2
num_workers = 8
out_dim = 5
init_lr = 3e-4
warmup_factor = 10

warmup_epo = 1
n_epochs = 10 if DEBUG else 30
df_train = df_train.sample(100).reset_index(drop=True) if DEBUG else df_train

device = torch.device('cuda')

print(TRAINING_DATA_FOLDER_PATH)
print(TESTING_DATA_FOLDER_PATH)


import os

device1 = "cuda" if torch.cuda.is_available() else "cpu"
nprocs = torch.cuda.device_count() if device1 == "cuda" else 1

master_addr = os.environ.get("MASTER_ADDR", "127.0.0.1")
master_port = os.environ.get("MASTER_PORT", "6006")
global_rank = int(os.environ.get("RANK", -1))
local_rank = int(os.environ.get("LOCAL_RANK", -1))
world_size = int(os.environ.get("WORLD_SIZE", nprocs))

9464
1052
/teamspace/studios/this_studio/Prostate_Cancer_Grade_Assessment/dataset/train-tiled-prostate-36x256x256
/teamspace/studios/this_studio/Prostate_Cancer_Grade_Assessment/dataset/test-tiled-prostate-36x256x256


In [7]:
DATASET_FOLDER_PATH

'/teamspace/studios/this_studio/Prostate_Cancer_Grade_Assessment/dataset'

# Create Folds

In [8]:
skf = StratifiedKFold(5, shuffle=True, random_state=42)
df_train['fold'] = -1
for i, (train_idx, valid_idx) in enumerate(skf.split(df_train, df_train['isup_grade'])):
    df_train.loc[valid_idx, 'fold'] = i
    
print(len(df_train))
df_train.head()

9464


,image_id,data_provider,isup_grade,gleason_score,is_present,fold
0,0005f7aaab2800f6170c399693a96917,karolinska,0,0+0,1,4
1,000920ad0b612851f8e01bcc880d9b3d,karolinska,0,0+0,1,4
2,001c62abd11fa4b57bf7a6c603a11bb9,karolinska,4,4+4,1,0
3,002a4db09dad406c85505a00fb6f6144,karolinska,0,0+0,1,1
4,003046e27c8ead3e3db155780dc5498e,karolinska,1,3+3,1,0


# Model

In [9]:
pretrained_model = {
    'efficientnet-b0': '/teamspace/studios/this_studio/Prostate_Cancer_Grade_Assessment/pre_trained_models/efficientnet-b0-08094119.pth'
}

In [10]:
class enetv2(nn.Module):
    def __init__(self, backbone, out_dim):
        super(enetv2, self).__init__()
        # self.enet = enet.EfficientNet.from_name(backbone)
        # self.enet.load_state_dict(torch.load(pretrained_model[backbone]))
        self.enet = enet.EfficientNet.from_pretrained('efficientnet-b0')

        self.myfc = nn.Linear(self.enet._fc.in_features, out_dim)
        self.enet._fc = nn.Identity()

    def extract(self, x):
        return self.enet(x)

    def forward(self, x):
        x = self.extract(x)
        x = self.myfc(x)
        return x

# Dataset

In [11]:

def get_tiles(img, mode=0):
        result = []
        h, w, c = img.shape
        pad_h = (tile_size - h % tile_size) % tile_size + ((tile_size * mode) // 2)
        pad_w = (tile_size - w % tile_size) % tile_size + ((tile_size * mode) // 2)

        img2 = np.pad(img,[[pad_h // 2, pad_h - pad_h // 2], [pad_w // 2,pad_w - pad_w//2], [0,0]], constant_values=255)
        img3 = img2.reshape(
            img2.shape[0] // tile_size,
            tile_size,
            img2.shape[1] // tile_size,
            tile_size,
            3
        )

        img3 = img3.transpose(0,2,1,3,4).reshape(-1, tile_size, tile_size,3)
        n_tiles_with_info = (img3.reshape(img3.shape[0],-1).sum(1) < tile_size ** 2 * 3 * 255).sum()
        if len(img3) < n_tiles:
            img3 = np.pad(img3,[[0,n_tiles-len(img3)],[0,0],[0,0],[0,0]], constant_values=255)
        idxs = np.argsort(img3.reshape(img3.shape[0],-1).sum(-1))[:n_tiles]
        img3 = img3[idxs]
        for i in range(len(img3)):
            result.append({'img':img3[i], 'idx':i})
        return result, n_tiles_with_info >= n_tiles
    
def create_tiles_object_from_images(folder_path, list_tile_img_files, list_tile_mask_files=[], include_mask=False):
        tiles = []

        if include_mask:
            for i, (tile_img_file, tile_mask_file) in enumerate(zip(list_tile_img_files, list_tile_mask_files)):
                tile_img = np.asarray(Image.open(os.path.join(folder_path, 'images', tile_img_file)))
                tile_mask = np.asarray(Image.open(os.path.join(folder_path, 'masks', tile_mask_file)))
                tiles.append({'img': tile_img, 'mask': tile_mask, 'idx': i})
        else:
            for i, tile_img_file in enumerate(list_tile_img_files):
                tile_img = np.asarray(Image.open(os.path.join(folder_path, 'images', tile_img_file)))
                tiles.append({'img': tile_img, 'idx': i})
        
        return tiles

class PANDADataset(Dataset):
    def __init__(self,
                 folder_path,
                 df,
                 image_size,
                 n_tiles=n_tiles,
                 tile_mode=0,
                 rand=False,
                 transform=None,
                ):
        self.folder_path = folder_path
        self.df = df.reset_index(drop=True)
        self.image_size = image_size
        self.n_tiles = n_tiles
        self.tile_mode = tile_mode
        self.rand = rand
        self.transform = transform

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, index):
        row = self.df.iloc[index]
        img_id = row.image_id
        
        # tiff_file = os.path.join(IMAGES_FOLDER, f'{img_id}.tiff')
        # image = skimage.io.MultiImage(tiff_file)[1]
        # tiles, _ = get_tiles(image, self.tile_mode)
        list_idx = list(range(0, n_tiles))
        list_tile_img_files = [f'{img_id}_{str(i)}.png' for i in list_idx]
        tiles = create_tiles_object_from_images(self.folder_path, list_tile_img_files, include_mask=False)

        if self.rand:
            idxes = np.random.choice(list(range(self.n_tiles)), self.n_tiles, replace=False)
        else:
            idxes = list(range(self.n_tiles))

        n_row_tiles = int(np.sqrt(self.n_tiles))
        images = np.zeros((image_size * n_row_tiles, image_size * n_row_tiles, 3))
        for h in range(n_row_tiles):
            for w in range(n_row_tiles):
                i = h * n_row_tiles + w
    
                if len(tiles) > idxes[i]:
                    this_img = tiles[idxes[i]]['img']
                else:
                    this_img = np.ones((self.image_size, self.image_size, 3)).astype(np.uint8) * 255
                this_img = 255 - this_img
                if self.transform is not None:
                    this_img = self.transform(image=this_img)['image']
                h1 = h * image_size
                w1 = w * image_size
                images[h1:h1+image_size, w1:w1+image_size] = this_img

        if self.transform is not None:
            images = self.transform(image=images)['image']
        images = images.astype(np.float32)
        images /= 255
        images = images.transpose(2, 0, 1)

        label = np.zeros(5).astype(np.float32)
        label[:row.isup_grade] = 1.
        return torch.tensor(images), torch.tensor(label)

# Augmentations

In [12]:
transforms_train = albumentations.Compose([
    albumentations.Transpose(p=0.5),
    albumentations.VerticalFlip(p=0.5),
    albumentations.HorizontalFlip(p=0.5),
])
transforms_val = albumentations.Compose([])

In [13]:
# dataset_show = PANDADataset(TESTING_DATA_FOLDER_PATH, df_test, image_size, n_tiles, 0, transform=None)
# from pylab import rcParams
# rcParams['figure.figsize'] = 20,10
# for i in range(2):
#     f, axarr = plt.subplots(1,5)
#     for p in range(5):
#         idx = np.random.randint(0, len(dataset_show))
#         img, label = dataset_show[idx]
#         axarr[p].imshow(1. - img.transpose(0, 1).transpose(1,2).squeeze())
#         axarr[p].set_title(str(sum(label)))

# Loss

In [14]:
criterion = nn.BCEWithLogitsLoss()

# Scheduler

In [15]:
from torch.optim.lr_scheduler import _LRScheduler
from torch.optim.lr_scheduler import ReduceLROnPlateau


class GradualWarmupScheduler(_LRScheduler):
    """ Gradually warm-up(increasing) learning rate in optimizer.
    Proposed in 'Accurate, Large Minibatch SGD: Training ImageNet in 1 Hour'.

    Args:
        optimizer (Optimizer): Wrapped optimizer.
        multiplier: target learning rate = base lr * multiplier if multiplier > 1.0. if multiplier = 1.0, lr starts from 0 and ends up with the base_lr.
        total_epoch: target learning rate is reached at total_epoch, gradually
        after_scheduler: after target_epoch, use this scheduler(eg. ReduceLROnPlateau)
    """

    def __init__(self, optimizer, multiplier, total_epoch, after_scheduler=None):
        self.multiplier = multiplier
        if self.multiplier < 1.:
            raise ValueError('multiplier should be greater thant or equal to 1.')
        self.total_epoch = total_epoch
        self.after_scheduler = after_scheduler
        self.finished = False
        super(GradualWarmupScheduler, self).__init__(optimizer)

    def get_lr(self):
        if self.last_epoch > self.total_epoch:
            if self.after_scheduler:
                if not self.finished:
                    self.after_scheduler.base_lrs = [base_lr * self.multiplier for base_lr in self.base_lrs]
                    self.finished = True
                return self.after_scheduler.get_lr()
            return [base_lr * self.multiplier for base_lr in self.base_lrs]

        if self.multiplier == 1.0:
            return [base_lr * (float(self.last_epoch) / self.total_epoch) for base_lr in self.base_lrs]
        else:
            return [base_lr * ((self.multiplier - 1.) * self.last_epoch / self.total_epoch + 1.) for base_lr in self.base_lrs]

    def step_ReduceLROnPlateau(self, metrics, epoch=None):
        if epoch is None:
            epoch = self.last_epoch + 1
        self.last_epoch = epoch if epoch != 0 else 1  # ReduceLROnPlateau is called at the end of epoch, whereas others are called at beginning
        if self.last_epoch <= self.total_epoch:
            warmup_lr = [base_lr * ((self.multiplier - 1.) * self.last_epoch / self.total_epoch + 1.) for base_lr in self.base_lrs]
            for param_group, lr in zip(self.optimizer.param_groups, warmup_lr):
                param_group['lr'] = lr
        else:
            if epoch is None:
                self.after_scheduler.step(metrics, None)
            else:
                self.after_scheduler.step(metrics, epoch - self.total_epoch)

    def step(self, epoch=None, metrics=None):
        if type(self.after_scheduler) != ReduceLROnPlateau:
            if self.finished and self.after_scheduler:
                if epoch is None:
                    self.after_scheduler.step(None)
                else:
                    self.after_scheduler.step(epoch - self.total_epoch)
                self._last_lr = self.after_scheduler.get_lr()
            else:
                return super(GradualWarmupScheduler, self).step(epoch)
        else:
            self.step_ReduceLROnPlateau(metrics, epoch)

# Train and Val

In [16]:
def train_epoch(loader, optimizer):

    model.train()
    train_loss = []
    bar = tqdm(loader)
    for (data, target) in bar:
        
        data, target = data.to(device), target.to(device)
        loss_func = criterion
        optimizer.zero_grad()
        logits = model(data)
        loss = loss_func(logits, target)
        loss.backward()
        optimizer.step()

        loss_np = loss.detach().cpu().numpy()
        train_loss.append(loss_np)
        smooth_loss = sum(train_loss[-100:]) / min(len(train_loss), 100)
        bar.set_description('loss: %.5f, smth: %.5f' % (loss_np, smooth_loss))
    return train_loss


def val_epoch(loader, get_output=False):

    model.eval()
    val_loss = []
    LOGITS = []
    PREDS = []
    TARGETS = []

    with torch.no_grad():
        for (data, target) in tqdm(loader):
            data, target = data.to(device), target.to(device)
            logits = model(data)

            loss = criterion(logits, target)

            pred = logits.sigmoid().sum(1).detach().round()
            LOGITS.append(logits)
            PREDS.append(pred)
            TARGETS.append(target.sum(1))

            val_loss.append(loss.detach().cpu().numpy())
        val_loss = np.mean(val_loss)

    LOGITS = torch.cat(LOGITS).cpu().numpy()
    PREDS = torch.cat(PREDS).cpu().numpy()
    TARGETS = torch.cat(TARGETS).cpu().numpy()
    acc = (PREDS == TARGETS).mean() * 100.
    
    qwk = cohen_kappa_score(PREDS, TARGETS, weights='quadratic')
    qwk_k = cohen_kappa_score(PREDS[df_valid['data_provider'] == 'karolinska'], df_valid[df_valid['data_provider'] == 'karolinska'].isup_grade.values, weights='quadratic')
    qwk_r = cohen_kappa_score(PREDS[df_valid['data_provider'] == 'radboud'], df_valid[df_valid['data_provider'] == 'radboud'].isup_grade.values, weights='quadratic')
    print('qwk', qwk, 'qwk_k', qwk_k, 'qwk_r', qwk_r)

    if get_output:
        return LOGITS
    else:
        return val_loss, acc, qwk

import copy
PREDS_1 = []
TARGETS_1 = []

def test_epoch(model, loader, get_output=False):

    model.eval()
    test_loss = []
    LOGITS = []
    PREDS = []
    TARGETS = []

    with torch.no_grad():
        for (data, target) in tqdm(loader):
            data, target = data.to(device), target.to(device)
            logits = model(data)

            loss = criterion(logits, target)

            pred = logits.sigmoid().sum(1).detach().round()
            LOGITS.append(logits)
            PREDS.append(pred)
            TARGETS.append(target.sum(1))

            test_loss.append(loss.detach().cpu().numpy())
        test_loss = np.mean(test_loss)

    PREDS_1 = copy.copy(PREDS)
    TARGETS_1 = copy.copy(TARGETS)

    LOGITS = torch.cat(LOGITS).cpu().numpy()
    PREDS = torch.cat(PREDS).cpu().numpy()
    TARGETS = torch.cat(TARGETS).cpu().numpy()
    print(PREDS)
    print(TARGETS)
    
    # PREDS_1 = copy.copy(PREDS)
    # TARGETS_1 = copy.copy(TARGETS)
    
    acc = (PREDS == TARGETS).mean() * 100.
    
    qwk = cohen_kappa_score(PREDS, TARGETS, weights='quadratic')
    qwk_k = cohen_kappa_score(PREDS[df_test['data_provider'] == 'karolinska'], df_test[df_test['data_provider'] == 'karolinska'].isup_grade.values, weights='quadratic')
    qwk_r = cohen_kappa_score(PREDS[df_test['data_provider'] == 'radboud'], df_test[df_test['data_provider'] == 'radboud'].isup_grade.values, weights='quadratic')
    print('qwk', qwk, 'qwk_k', qwk_k, 'qwk_r', qwk_r)

    if get_output:
        return LOGITS
    else:
        return PREDS_1, TARGETS_1, test_loss, acc, qwk


# Create Dataloader & Model & Optimizer

In [22]:
train_idx = np.where((df_train['fold'] != fold))[0]
valid_idx = np.where((df_train['fold'] == fold))[0]

df_this  = df_train.loc[train_idx]
df_valid = df_train.loc[valid_idx]
df_test = df_test.loc[:]

dataset_train = PANDADataset(TRAINING_DATA_FOLDER_PATH, df_this, image_size, n_tiles, transform=transforms_train)
dataset_valid = PANDADataset(TRAINING_DATA_FOLDER_PATH, df_valid, image_size, n_tiles, transform=transforms_val)
dataset_test = PANDADataset(TESTING_DATA_FOLDER_PATH, df_test, image_size, n_tiles, transform=None)

train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size, sampler=RandomSampler(dataset_train), num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(dataset_valid, batch_size=batch_size, sampler=SequentialSampler(dataset_valid), num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(dataset_test, batch_size=batch_size, sampler=SequentialSampler(dataset_test), num_workers=num_workers)

model = enetv2(enet_type, out_dim=out_dim)

# # Using trained model from 24th early morning (not that helpful but still)
# trained_fold_model_file_path: str = '/teamspace/studios/this_studio/04_24_2024_05_38_13_final_fold0.pth'
# # model.load_state_dict(torch.load(trained_fold_model_file_path))
# # model.load_state_dict(torch.load(trained_fold_model_file_path, weights_only=True))
# fold_weights = torch.load(trained_fold_model_file_path)

# if torch.cuda.device_count() > 1:
#   print("Let's use", torch.cuda.device_count(), "GPUs!")
#   # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
#   model = nn.DataParallel(model, device_ids=[0,1,2,3])

model = model.to(device)

optimizer = optim.Adam(model.parameters(), lr=init_lr/warmup_factor)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, n_epochs-warmup_epo)
scheduler = GradualWarmupScheduler(optimizer, multiplier=warmup_factor, total_epoch=warmup_epo, after_scheduler=scheduler_cosine)

print(len(dataset_train), len(dataset_valid), len(dataset_test))

Loaded pretrained weights for efficientnet-b0
7571 1893 1052


# Run Training

In [18]:
from datetime import datetime

dt_string = datetime.now().strftime("%m_%d_%Y_%H_%M_%S")
print(dt_string)

04_28_2024_03_50_50


In [19]:
# qwk_max = 0.
# best_file = f'{dt_string}_best_fold{fold}.pth'
# for epoch in range(1, n_epochs+1):
#     print(time.ctime(), 'Epoch:', epoch)
#     scheduler.step(epoch-1)

#     train_loss = train_epoch(train_loader, optimizer)
#     val_loss, acc, qwk = val_epoch(valid_loader)

#     content = time.ctime() + ' ' + f'Epoch {epoch}, lr: {optimizer.param_groups[0]["lr"]:.7f}, train loss: {np.mean(train_loss):.5f}, val loss: {np.mean(val_loss):.5f}, acc: {(acc):.5f}, qwk: {(qwk):.5f}'
#     print(content)
#     with open(f'log_{dt_string}.txt', 'a') as appender:
#         appender.write(content + '\n')

#     if qwk > qwk_max:
#         print('score2 ({:.6f} --> {:.6f}).  Saving model ...'.format(qwk_max, qwk))
#         torch.save(model.state_dict(), best_file)
#         qwk_max = qwk

# torch.save(model.state_dict(), os.path.join(f'{dt_string}_final_fold{fold}.pth'))

# Run Inference Only

In [23]:
trained_model_file_path: str = '/teamspace/studios/this_studio/trained_models/effnet_b0_LB_0.86.pth'
model.load_state_dict(torch.load(trained_model_file_path, map_location=torch.device('cuda')))

preds_1, targets_1, test_loss, acc, qwk = test_epoch(model=model, loader=test_loader)



/tmp/ipykernel_49635/2349884661.py:74: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for (data, target) in tqdm(loader):


  0%|          | 0/526 [00:00<?, ?it/s]

[3. 3. 3. ... 3. 3. 3.]
[3. 1. 4. ... 4. 3. 1.]
qwk 0.014613699135250457 qwk_k 0.0097251619311165 qwk_r 0.022587378357608823
